In [2]:
import pandas as pd
test_ds = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_raw.csv', index_col=None)

In [3]:
test_ds.head(1)

,topic,stance,sentence,q_score,word_count
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations isn't mandatory sin...,0.714195,11


In [4]:
#train_df = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\train_complete.csv", index_col=None)
#dev_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\dev_complete.csv", index_col=None)
test_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\test_complete.csv", index_col=None)
# to be able to compare, we take only arguments with positive single label
test_ds_labeled = test_df[test_df['label'] == 1]
test_ds_labeled = test_ds_labeled.sample(frac=1).drop_duplicates(subset='arg_id')

# clustering with TOP2Vec

In [5]:
test_ds_grouped = test_ds.groupby(['topic', 'stance']).agg({
    'sentence': lambda x: [i for i in x]
}).reset_index()

In [ ]:
pd.set_option('display.max_colwidth', None)
test_ds_grouped.head(1)

In [8]:
test_ds_grouped.to_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\test_grouped.csv", index=False)

In [48]:
from top2vec import Top2Vec

# good results were shown from paraphrase-multilingual-MiniLM-L12-v2, distiluse-base-multilingual-cased-v2

orig_df = test_ds_grouped

#test_ds_grouped = test_ds_grouped.head(2)

results_df = pd.DataFrame({})
for index, row in test_ds_grouped.iterrows():
    corpus = list(row['sentence'])
    model = Top2Vec(corpus,  embedding_model='distiluse-base-multilingual-cased-v2', hdbscan_args={"min_cluster_size":5}, min_count =7)
    topic_sizes, topic_nums = model.get_topic_sizes()
    topic_words, word_scores, topic_nums = model.get_topics(len(topic_nums))
    aspect_docs = []
    aspect_docs_scores = []
    sub_topic_ids = []
    topics = []
    stances = []
    topics_size = []
    topic_word = []
    word_score = []
    for t_s, t_n in zip(topic_sizes, topic_nums):
        sub_topic_ids.append(t_n)
        documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=t_n, num_docs=t_s)
        aspect_docs.append(documents)
        aspect_docs_scores.append(document_scores)
        topics.append(row['topic'])
        stances.append(row['stance'])
        topics_size.append(topic_sizes[t_n])
        topic_word.append(topic_words[t_n])
        word_score.append(word_scores[t_n])
    results_df = pd.concat([results_df, pd.DataFrame({
        "topic": topics, "stance": stances, "aspect_id": sub_topic_ids, "aspect_docs": aspect_docs, "aspect_docs_scores": aspect_docs_scores, "topic_size": topics_size, "topic_words":topic_word, "word_scores": word_score})])
results_df['centroid'] = results_df['aspect_docs'].apply(lambda row: row[0] )

2022-02-03 14:17:35,660 - top2vec - INFO - Pre-processing documents for training
C:\Users\Bilal\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-02-03 14:17:35,670 - top2vec - INFO - Downloading distiluse-base-multilingual-cased-v2 model
2022-02-03 14:17:43,564 - top2vec - INFO - Creating joint document/word embedding
2022-02-03 14:17:44,551 - top2vec - INFO - Creating lower dimension embedding of documents
2022-02-03 14:17:46,915 - top2vec - INFO - Finding dense areas of documents
2022-02-03 14:17:46,922 - top2vec - INFO - Finding topics
2022-02-03 14:17:46,996 - top2vec - INFO - Pre-processing documents for training
C:\Users\Bilal\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_featur

In [ ]:
results_df.head(2)

In [128]:
test_ds_grouped.head(2)

,topic,stance,sentence,topics_sizes,topic_nums,topic_words,word_scores
0,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory si...,"[59, 34, 23]","[0, 1, 2]","[[vaccinations, vaccines, vaccination, vaccine...","[[0.5526118, 0.5518729, 0.5435168, 0.5362369, ..."
1,Routine child vaccinations should be mandatory,1,[ Routine childhood vaccinations should be ma...,"[75, 27, 26, 22, 17, 9]","[0, 1, 2, 3, 4, 5]","[[vaccinations, vaccines, vaccination, vaccine...","[[0.55734587, 0.55435467, 0.55224234, 0.542876..."


In [109]:
results_df.columns

Index(['topic', 'stance', 'aspect_id', 'aspect_docs', 'aspect_docs_scores',
       'topic_size', 'topic_words', 'word_scores', 'centroid'],
      dtype='object')

In [51]:
len(results_df)

23

In [50]:
pd.set_option('display.max_colwidth', None)
results_df[['topic', 'stance', 'centroid', 'aspect_id', 'topic_size']]

,topic,stance,centroid,aspect_id,topic_size
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,0,80
1,Routine child vaccinations should be mandatory,-1,"It shouldn't be mandatory , every parent should be able to decide whether to vaccinate their childs or not.",1,36
0,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent the disease,0,81
1,Routine child vaccinations should be mandatory,1,Child vaccination saves lives and keeps children from suffering from preventable illness.,1,25
2,Routine child vaccinations should be mandatory,1,"Routine childhood vaccines should be mandatory, because they prevent any disease in the child, and thus ensure a healthy future",2,21
3,Routine child vaccinations should be mandatory,1,vaccines are antibodies that we need to avoid disease,3,15
4,Routine child vaccinations should be mandatory,1,it should be mandatory because sometimes children are very prone to diseases,4,15
5,Routine child vaccinations should be mandatory,1,yes as they protect children from life threatening and highly infectious diseases,5,10
6,Routine child vaccinations should be mandatory,1,protecting infants must be a priority for all,6,9
0,Social media platforms should be regulated by the government,-1,Social media platform should not be regulated by the government because it undermines freedom of speech,0,59


In [52]:
results_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\top2vec_groups_test.csv', index=False)

In [21]:
import pandas as pd
results_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\top2vec_groups_test.csv', index_col=None)

In [24]:
results_df.head(2)

topic  stance  aspect_id  \
0  Routine child vaccinations should be mandatory      -1          0   
1  Routine child vaccinations should be mandatory      -1          1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [131]:
# take n senetences closest to centroid
candidates = []
candidates_cluster_score = []
aspects_ids = []
topic_size = []
CLOSEST_TO_CENTROID_N = 10
for i, row in results_df.iterrows():
    candidates.extend(row['aspect_docs'][:CLOSEST_TO_CENTROID_N])
    candidates_cluster_score.extend(row['aspect_docs_scores'][:CLOSEST_TO_CENTROID_N])
    aspects_ids.extend([row["aspect_id"]] * len(row['aspect_docs_scores'][:CLOSEST_TO_CENTROID_N]))
    topic_size.extend([row["topic_size"]] * len(row['aspect_docs_scores'][:CLOSEST_TO_CENTROID_N]))
    
print(len(candidates))

226


In [132]:
# add q_score & # of words
candidates_df = pd.DataFrame({"sentence": candidates, "cluster_score": candidates_cluster_score, "aspects_id": aspects_ids, "topic_size": topic_size})
candidates_df = pd.merge(candidates_df, test_ds, on='sentence')
# basic filtering
candidates_df = candidates_df[candidates_df['cluster_score'] > 0.7]
candidates_df = candidates_df[candidates_df['q_score'] > 0.5]
mask = (candidates_df['word_count'] > 5) & (candidates_df['word_count'] < 20)
candidates_df = candidates_df.loc[mask]
print(len(candidates_df))

127


In [133]:
# sort by q_score & # of words, then shortlist to n sentences
# try ["cluster_score", "q_score","word_count"]
# get top n sentences per topic/stance and check if enough 
candidates_count_df = candidates_df.groupby(by=["topic", "stance", "aspects_id"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["cluster_score", "q_score","word_count"], ascending=(False, False, True)).head(5)).reset_index(drop=True)

In [134]:
len(candidates_count_df)

97

In [135]:
candidates_count_df.groupby(by=["topic", "stance", "aspects_id"]).size()

topic                                                         stance  aspects_id
Routine child vaccinations should be mandatory                -1      0             5
                                                                      1             5
                                                               1      0             5
                                                                      1             5
                                                                      2             5
                                                                      3             5
                                                                      4             5
                                                                      5             2
                                                                      6             4
Social media platforms should be regulated by the government  -1      0             4
                                                           

In [142]:
# Feed the candatates to matching model and check results
candidates_count_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\candidates_clustering_test.csv', index=False)

In [8]:
import pandas as pd
candidates_count_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\candidates_clustering_test.csv', index_col=None)

In [11]:
# try getting top only with clustering 
top_df = candidates_count_df.groupby(by=["topic", "stance", "aspects_id"]).head(1).reset_index(drop=True)

In [12]:
len(top_df)

22

In [14]:
pd.set_option('display.max_colwidth', None)
top_df[["topic", "stance", "sentence",  "cluster_score", "q_score", "word_count", "topic_size"]].head(25)

,topic,stance,sentence,cluster_score,q_score,word_count,topic_size
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,0.933753,0.871068,17,80
1,Routine child vaccinations should be mandatory,-1,"It shouldn't be mandatory , every parent should be able to decide whether to vaccinate their childs or not.",0.839476,0.780102,19,36
2,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent the disease,0.944496,0.854549,10,81
3,Routine child vaccinations should be mandatory,1,Child vaccination saves lives and keeps children from suffering from preventable illness.,0.846536,0.936823,12,25
4,Routine child vaccinations should be mandatory,1,vaccines must be compulsory for children because in this way we prevent later diseases in infants,0.914108,0.922973,16,21
5,Routine child vaccinations should be mandatory,1,vaccines are antibodies that we need to avoid disease,0.847310,0.797935,9,15
6,Routine child vaccinations should be mandatory,1,it should be mandatory because sometimes children are very prone to diseases,0.864239,0.809383,12,15
7,Routine child vaccinations should be mandatory,1,yes as they protect children from life threatening and highly infectious diseases,0.811347,0.728977,12,10
8,Routine child vaccinations should be mandatory,1,protecting infants must be a priority for all,0.861261,0.783763,8,9
9,Social media platforms should be regulated by the government,-1,Social media platform should not be regulated by the government because it undermines freedom of speech,0.953973,0.966574,16,59


In [3]:
import pandas as pd
test_ds_labeled_counts = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_ds_labeled_counts.csv", index_col=None)

In [15]:
test_ds_labeled_counts.head(40)

,topic,stance,key_point,counts
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations or their side effects are dangerous,53
1,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,21
2,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,21
3,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary to keep children healthy,11
4,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable diseases,54
5,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent virus disease spreading,30
6,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,27
7,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to protect others,21
8,Social media platforms should be regulated by the government,-1,Social media regulation harm freedom of speech and other democratic rights,49
9,Social media platforms should be regulated by the government,-1,Social media regulation harms privacy,15


In [16]:
import rouge
# evaluate to ground truth using rouge
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [19]:
# comparing test_ds_labeled_counts with counst_df
# for every topic, stance => compare predicted kp with every labeled kp and take max score
import numpy as np
from itertools import product
unique_topics = test_ds_labeled['topic'].unique()
unique_stances = [-1, 1]

topics = []
stances = []
scores = []

for t in unique_topics:
    for s in unique_stances:
        test_df = test_ds_labeled_counts[test_ds_labeled_counts['topic'] == t]
        test_df = test_df[test_df['stance'] == s]
        res_df = top_df[top_df['topic'] == t]
        res_df = res_df[res_df['stance'] == s]
        
        unique_predicted_kps = res_df['sentence'].unique()
        unique_labeled_kps = test_df['key_point'].unique()
        kp_scores = []
        for p_kp, l_kp in product(unique_predicted_kps, unique_labeled_kps) :
                score = evaluator.get_scores(p_kp, l_kp)
                kp_scores.append(score['rouge-l']['f'])    

        topics.append(t)
        stances.append(s)
        scores.append(np.mean(kp_scores))
        
    
results_df = pd.DataFrame({"topic": topics, "stance": stances,  "r_scores": scores})
    

In [20]:
results_df.head(6)

,topic,stance,r_scores
0,Routine child vaccinations should be mandatory,-1,0.270400
1,Routine child vaccinations should be mandatory,1,0.313076
2,Social media platforms should be regulated by the government,-1,0.235033
3,Social media platforms should be regulated by the government,1,0.295791
4,The USA is a good country to live in,-1,0.177127
5,The USA is a good country to live in,1,0.192985


In [ ]:


# summarize results at the end with experiments vs evaluation result
# evaluation result will be for every topic/stance & experments, rouge score , aspect coverage ratio (aspects predicted/all ground aspects)
# divide experiments to:

Seems basic filtering applied to all
- getting sentences
- quality threshold
- word length 

Candidate selection:
    - only basic filtering
    - advanced filtering (cross similarity top match + redundancy removal only )
    - PR with similarity matching filtering technique
    - Cluster filtering 

Generation
- Top from PR alone
- Top from cross similarity alone
- Top from cluster filtering
- candidates (either technique) + task model matching overlap ratio  
- candidates (either technique) + task model matching top macthes  


Experiments:
    - cluster filtering only => done
    - advanced filtering only 
    - PR only => done
    - cluster + task modeling overlap ratio
    - basic + task modeling overlap ratio
    - advanced filtering + task model matching top macthes  
    
Summary of all